In [2]:
import sys; sys.version

'3.11.10 | packaged by conda-forge | (main, Sep 30 2024, 17:56:25) [GCC 13.3.0]'

In [3]:
import pandas as pd
import glob

In [4]:
!du -hs data/*

77M	data/test_news.csv
241M	data/train.csv
252M	data/train_raw


In [5]:
%%time
bloc_types_src = {
  'Общество/Россия': 1,
  'Экономика': 4,
  'Силовые структуры': 37, 
  'Бывший СССР': 3,
  'Спорт': 8,
  'Забота о себе': 87,
  'Туризм/Путешествия': 48, 
  'Наука и техника': 5,
}
bloc_types = dict(zip(bloc_types_src.values(), bloc_types_src.keys()))
train_df = pd.DataFrame()
for filepath in glob.glob('data/train_raw/*.csv'):
    bloc = filepath.split('/')[-1].split('_')[0].replace('bloc', '')
    if not bloc.isnumeric(): continue
    name = bloc_types.get(int(bloc), None)
    assert name is not None, name
    _df = pd.read_csv(filepath)
    _df['news_topic'] = name
    train_df = pd.concat([train_df, _df])
train_df.shape

CPU times: user 3.23 s, sys: 378 ms, total: 3.61 s
Wall time: 3.81 s


(77500, 18)

In [8]:
train_df.drop(columns='Unnamed: 0', inplace=True)

In [13]:
train_df['target'] = train_df.news_topic.map(bloc_types_src.get)
train_df['target'].value_counts()

target
48    10000
8     10000
3     10000
5     10000
87    10000
37    10000
1     10000
4      7500
Name: count, dtype: int64

In [14]:
train_df.columns

Index(['docid', 'url', 'title', 'modified', 'lastmodtime', 'type', 'domain',
       'status', 'part', 'bloc', 'tags', 'image_url', 'pubdate', 'text',
       'rightcol', 'snippet', 'news_topic', 'target'],
      dtype='object')

In [15]:
train_df.isna().sum()

docid            0
url              0
title            0
modified         0
lastmodtime      0
type             0
domain           0
status           0
part             0
bloc             0
tags             0
image_url      159
pubdate          0
text             0
rightcol         0
snippet         21
news_topic       0
target           0
dtype: int64

In [16]:
clear_train_df = train_df[['target', 'news_topic', 'text']]
clear_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   target      77500 non-null  int64 
 1   news_topic  77500 non-null  object
 2   text        77500 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.4+ MB


In [18]:
clear_train_df.to_csv('data/train_news.csv', index=False)
!du -hs data/train_news.csv

177M	data/train_news.csv
